In [1]:
import pandas as pd

In [38]:
df = pd.read_csv('resultsetv2.csv', sep='#')

In [39]:
# pd.set_option('display.max_rows', None)

In [49]:
df

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_type_desc,spec_itemid,isolate_num,org_itemid,ab_itemid
0,3,145834,NaN,NaN,NaN,76.52,M,Unspecified septicemia,STOOL,70064.0,NaN,NaN,NaN
1,3,145834,NaN,NaN,NaN,76.52,M,Unspecified septicemia,URINE,70079.0,NaN,NaN,NaN
2,3,145834,NaN,NaN,NaN,76.52,M,Unspecified septicemia,BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),70011.0,NaN,NaN,NaN
3,3,145834,NaN,NaN,NaN,76.52,M,Unspecified septicemia,BLOOD CULTURE,70012.0,NaN,NaN,NaN
4,3,145834,NaN,NaN,NaN,76.52,M,Unspecified septicemia,CATHETER TIP-IV,70023.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
461101,99991,151118,NaN,NaN,NaN,47.72,M,Unspecified septicemia,BLOOD CULTURE,70012.0,NaN,NaN,NaN
461102,99991,151118,NaN,NaN,NaN,47.72,M,Unspecified septicemia,BLOOD CULTURE,70012.0,NaN,NaN,NaN
461103,99991,151118,NaN,NaN,NaN,47.72,M,Unspecified septicemia,URINE,70079.0,NaN,NaN,NaN
461104,99991,151118,NaN,NaN,NaN,47.72,M,Unspecified septicemia,BLOOD CULTURE,70012.0,NaN,NaN,NaN


# Exploratory analysis

## First query from MIMIC III (df)

In [5]:
# How many admissions
df['hadm_id'].nunique()

6313

In [7]:
# How many patients
df['subject_id'].nunique()

5542

In [8]:
# Gender of 5542 patients
gender_count = df.groupby(['subject_id', 'gender']).agg(count_gender=('gender', 'count')).reset_index()

In [10]:
gender_count.groupby(['gender']).agg(count_gender=('gender', 'count'))

,count_gender
gender,
F,2465
M,3077


### How many patients have positive culture?

In [11]:
### Lines with org_name null
df['org_name'].isna().sum()

194620

In [12]:
### Lines with org_name not null
df['org_name'].count()

266486

In [13]:
### Lines with org_name null for each hadm_id
count_orgname_to_hadmid = df.groupby('hadm_id').agg({'org_name': lambda x: x.isnull().sum()})

In [14]:
count_orgname_to_hadmid

,org_name
hadm_id,
100028,10
100074,9
100104,15
100117,30
100118,12
...,...
199900,28
199919,28
199929,28


In [15]:
### Dataframe removing lines containing org_name null
df_with_org_name_not_null = df[pd.notnull(df['org_name'])]
df_with_org_name_not_null

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_type_desc,spec_itemid,isolate_num,org_itemid,ab_itemid
5,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,SPUTUM,70062.0,1.0,80075.0,NaN
6,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,URINE,70079.0,1.0,80075.0,NaN
10,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,URINE,70079.0,1.0,80075.0,NaN
13,3,145834,GRAM NEGATIVE ROD(S),NaN,NaN,76.52,M,Unspecified septicemia,SWAB,70070.0,1.0,80058.0,NaN
14,3,145834,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",NaN,NaN,76.52,M,Unspecified septicemia,SWAB,70070.0,1.0,80155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
461085,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Severe sepsis,MRSA SCREEN,70091.0,1.0,80293.0,NaN
461088,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Severe sepsis,MRSA SCREEN,70091.0,1.0,80293.0,NaN
461096,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Unspecified septicemia,MRSA SCREEN,70091.0,1.0,80293.0,NaN
461097,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Unspecified septicemia,MRSA SCREEN,70091.0,1.0,80293.0,NaN


### Admissions with positive culture: 5090

In [16]:
df_with_org_name_not_null['hadm_id'].nunique()

5090

### Patients with positive culture: 4479

In [12]:
df_with_org_name_not_null['subject_id'].nunique()

4479

# SAVING NEW DATAFRAME
## POSITIVE_CULTURES

In [94]:
df_with_org_name_not_null.to_csv('positive_cultures.csv', index=False)

In [95]:
positive_cultures = df_with_org_name_not_null

In [96]:
type(positive_cultures)

pandas.core.frame.DataFrame

## CHANGES IN DATAFRAME

## DROP

### Deleting some diagnosis which doesn't fit in the analysis

In [98]:
positive_cultures = pd.read_csv('positive_cultures.csv')

In [99]:
# Checking the diagnoses
positive_cultures.groupby('diagnoses').agg(['count'])

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,spec_type_desc,spec_itemid,isolate_num,org_itemid,ab_itemid
,count,count,count,count,count,count,count,count,count,count,count,count
diagnoses,,,,,,,,,,,,
Herpetic septicemia,43,43,43,18,18,43,43,43,43,43,43,18
Methicillin resistant Staphylococcus aureus septicemia,3920,3920,3920,3450,3450,3920,3920,3920,3910,3920,3903,3450
Methicillin susceptible Staphylococcus aureus septicemia,15127,15127,15127,12531,12531,15127,15127,15127,15105,15127,15127,12531
"Obstetrical pyemic and septic embolism, antepartum condition or complication",13,13,13,10,10,13,13,13,13,13,13,10
Other septicemia due to gram-negative organisms,12315,12315,12315,10689,10689,12315,12315,12315,12315,12315,12143,10689
Other specified septicemias,7427,7427,7427,5991,5991,7427,7427,7427,7409,7427,7369,5991
Other staphylococcal septicemia,4977,4977,4977,4304,4304,4977,4977,4977,4970,4977,4967,4304
Pneumococcal septicemia [Streptococcus pneumoniae septicemia],1159,1159,1159,853,853,1159,1159,1159,1159,1159,1142,853


In [100]:
# Herpetic septicemia
positive_cultures = positive_cultures[positive_cultures.diagnoses != 'Herpetic septicemia']

In [101]:
# Septicemia due to anaerobes
positive_cultures = positive_cultures[positive_cultures.diagnoses != 'Septicemia due to anaerobes']

### Deleting some specimen types

In [102]:
# checking the specimen
positive_cultures.groupby('spec_type_desc').agg(['count'])

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
,count,count,count,count,count,count,count,count,count,count,count,count
spec_type_desc,,,,,,,,,,,,
ABSCESS,5050,5050,5050,4381,4381,5050,5050,5050,5050,5050,5026,4381
ASPIRATE,342,342,342,285,285,342,342,342,342,342,342,285
BILE,3251,3251,3251,2905,2905,3251,3251,3251,3251,3251,3197,2905
BIOPSY,114,114,114,99,99,114,114,114,114,114,114,99
BLOOD CULTURE,69813,69813,69813,58817,58817,69813,69813,69813,69813,69813,69247,58817
BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),1372,1372,1372,1009,1009,1372,1372,1372,1372,1372,1343,1009
BRONCHIAL BRUSH,27,27,27,16,16,27,27,27,27,27,27,16
BRONCHIAL BRUSH - PROTECTED,3,3,3,0,0,3,3,3,3,3,3,0


In [103]:
# BIOPSY
positive_cultures = positive_cultures.set_index("spec_type_desc")
positive_cultures = positive_cultures.drop("BIOPSY", axis=0)

In [104]:
# DIRECT ANTIGEN TEST FOR VARICELLA-ZOSTER VIRUS
positive_cultures = positive_cultures.drop("DIRECT ANTIGEN TEST FOR VARICELLA-ZOSTER VIRUS", axis=0)

In [105]:
# Direct Antigen Test for Herpes Simplex Virus Types 1 & 2
positive_cultures = positive_cultures.drop("Direct Antigen Test for Herpes Simplex Virus Types 1 & 2", axis=0)

In [106]:
# Influenza A/B by DFA
positive_cultures = positive_cultures.drop('Influenza A/B by DFA', axis=0)

In [107]:
# Isolate
positive_cultures = positive_cultures.drop('Isolate', axis=0)

In [108]:
# POSTMORTEM CULTURE
positive_cultures = positive_cultures.drop('POSTMORTEM CULTURE', axis=0)

In [109]:
positive_cultures = positive_cultures.drop('RAPID RESPIRATORY VIRAL ANTIGEN TEST', axis=0)

In [110]:
positive_cultures = positive_cultures.drop('Rapid Respiratory Viral Screen & Culture', axis=0)

In [111]:
# VARICELLA-ZOSTER CULTURE
positive_cultures = positive_cultures.drop('VARICELLA-ZOSTER CULTURE', axis=0)

In [112]:
# VIRAL CULTURE:R/O HERPES SIMPLEX VIRUS
positive_cultures = positive_cultures.drop('VIRAL CULTURE:R/O HERPES SIMPLEX VIRUS', axis=0)

In [113]:
positive_cultures = positive_cultures.drop('WORM', axis=0)

In [114]:
positive_cultures = positive_cultures.drop('XXX', axis=0)

In [115]:
positive_cultures = positive_cultures.reset_index()

## Rename / Replace

In [116]:
positive_cultures

,spec_type_desc,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
0,SPUTUM,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,70062.0,1.0,80075.0,NaN
1,URINE,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,70079.0,1.0,80075.0,NaN
2,URINE,3,145834,YEAST,NaN,NaN,76.52,M,Unspecified septicemia,70079.0,1.0,80075.0,NaN
3,SWAB,3,145834,GRAM NEGATIVE ROD(S),NaN,NaN,76.52,M,Unspecified septicemia,70070.0,1.0,80058.0,NaN
4,SWAB,3,145834,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",NaN,NaN,76.52,M,Unspecified septicemia,70070.0,1.0,80155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263428,MRSA SCREEN,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Severe sepsis,70091.0,1.0,80293.0,NaN
263429,MRSA SCREEN,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Severe sepsis,70091.0,1.0,80293.0,NaN
263430,MRSA SCREEN,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Unspecified septicemia,70091.0,1.0,80293.0,NaN
263431,MRSA SCREEN,99991,151118,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,NaN,NaN,47.72,M,Unspecified septicemia,70091.0,1.0,80293.0,NaN


In [117]:
positive_cultures.rename(columns = {'spec_type_desc':'spec_type'}, inplace = True) 

In [118]:
# Checking spec_type
positive_cultures.groupby('spec_type').agg(['count'])

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
,count,count,count,count,count,count,count,count,count,count,count,count
spec_type,,,,,,,,,,,,
ABSCESS,5050,5050,5050,4381,4381,5050,5050,5050,5050,5050,5026,4381
ASPIRATE,342,342,342,285,285,342,342,342,342,342,342,285
BILE,3251,3251,3251,2905,2905,3251,3251,3251,3251,3251,3197,2905
BLOOD CULTURE,69813,69813,69813,58817,58817,69813,69813,69813,69813,69813,69247,58817
BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),1372,1372,1372,1009,1009,1372,1372,1372,1372,1372,1343,1009
BRONCHIAL BRUSH,27,27,27,16,16,27,27,27,27,27,27,16
BRONCHIAL BRUSH - PROTECTED,3,3,3,0,0,3,3,3,3,3,3,0
BRONCHIAL WASHINGS,1588,1588,1588,1255,1255,1588,1588,1588,1588,1588,1531,1255


In [119]:
# Grouping by infection site
positive_cultures['spec_type'].replace({'BRONCHIAL WASHINGS': 'BRONCHOALVEOLAR LAVAGE', 
                           'FLUID RECEIVED IN BLOOD CULTURE BOTTLES': 'FLUID,OTHER',
                          'URINE,SUPRAPUBIC ASPIRATE' : 'URINE',
                          'URINE,KIDNEY':'URINE',
                          'BRONCHIAL BRUSH - PROTECTED' : 'BRONCHIAL BRUSH',
                          'BRONCHIAL BRUSH': 'BRONCHOALVEOLAR LAVAGE',
                         'THROAT CULTURE' : 'THROAT',
                         'THROAT FOR STREP': 'THROAT',
                         }, inplace=True)

In [120]:
# # Grouping by infection site
positive_cultures['spec_type'].replace({'BRONCHIAL BRUSH' : 'BRONCHOALVEOLAR LAVAGE',
                         'FLUID WOUND' : 'FLUID,OTHER'}, inplace=True)

In [121]:
positive_cultures.groupby('spec_type').agg(['count'])

,subject_id,hadm_id,org_name,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
,count,count,count,count,count,count,count,count,count,count,count,count
spec_type,,,,,,,,,,,,
ABSCESS,5050,5050,5050,4381,4381,5050,5050,5050,5050,5050,5026,4381
ASPIRATE,342,342,342,285,285,342,342,342,342,342,342,285
BILE,3251,3251,3251,2905,2905,3251,3251,3251,3251,3251,3197,2905
BLOOD CULTURE,69813,69813,69813,58817,58817,69813,69813,69813,69813,69813,69247,58817
BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),1372,1372,1372,1009,1009,1372,1372,1372,1372,1372,1343,1009
BRONCHOALVEOLAR LAVAGE,8707,8707,8707,6299,6299,8707,8707,8707,8707,8707,8604,6299
CATHETER TIP-IV,10007,10007,10007,9744,9744,10007,10007,10007,10007,10007,9983,9744
CSF;SPINAL FLUID,285,285,285,242,242,285,285,285,285,285,285,242


## Removing some org_name

In [122]:
positive_cultures.groupby('org_name').agg(['count'])

,spec_type,subject_id,hadm_id,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
,count,count,count,count,count,count,count,count,count,count,count,count
org_name,,,,,,,,,,,,
2ND ISOLATE,351,351,351,0,0,351,351,351,351,351,351,0
ABIOTROPHIA/GRANULICATELLA SPECIES,5,5,5,0,0,5,5,5,5,5,5,0
ACIDFAST BACILLI,44,44,44,0,0,44,44,44,44,44,44,0
ACINETOBACTER BAUMANNII,2448,2448,2448,2375,2375,2448,2448,2448,2448,2448,2448,2375
ACINETOBACTER BAUMANNII COMPLEX,1373,1373,1373,1347,1347,1373,1373,1373,1373,1373,1373,1347
...,...,...,...,...,...,...,...,...,...,...,...,...
VIBRIO SPECIES,5,5,5,0,0,5,5,5,5,5,5,0
VIRIDANS STREPTOCOCCI,576,576,576,233,233,576,576,576,576,576,576,233


In [50]:
# 2ND ISOLATE
positive_cultures = positive_cultures.set_index("org_name")
positive_cultures = positive_cultures.drop("2ND ISOLATE", axis=0)

In [51]:
positive_cultures = positive_cultures.drop("ADENOVIRUS", axis=0)

In [52]:
positive_cultures = positive_cultures.drop("AFB GROWN IN CULTURE; ADDITIONAL INFORMATION TO FOLLOW", axis=0)

In [53]:
positive_cultures = positive_cultures.drop("BACTERIA", axis=0)

In [54]:
positive_cultures = positive_cultures.drop("BASIDIOMYCETE", axis=0)

In [55]:
positive_cultures = positive_cultures.drop("BETA STREPTOCOCCUS", axis=0)

In [56]:
positive_cultures = positive_cultures.drop("CANCELLED", axis=0)

In [57]:
positive_cultures = positive_cultures.drop("CHAETOMIUM SPECIES", axis=0)

In [58]:
positive_cultures = positive_cultures.drop("CHILOMASTIX MESNILI", axis=0)

In [59]:
positive_cultures = positive_cultures.drop("CHLAMYDIA TRACHOMATIS", axis=0)

In [60]:
positive_cultures = positive_cultures.drop("DOLOSIGRANULUM PIGRUM", axis=0)

In [61]:
positive_cultures = positive_cultures.drop("ENTAMOEBA COLI", axis=0)

In [62]:
positive_cultures = positive_cultures.drop("FEW BLASTOCYSTIS HOMINIS", axis=0)

In [63]:
positive_cultures = positive_cultures.drop("HERPES SIMPLEX VIRUS TYPE 1", axis=0)

In [64]:
positive_cultures = positive_cultures.drop("HERPES SIMPLEX VIRUS TYPE 2", axis=0)

In [65]:
positive_cultures = positive_cultures.drop("MODERATE BLASTOCYSTIS HOMINIS", axis=0)

In [66]:
positive_cultures = positive_cultures.drop("MOLD", axis=0)

In [67]:
positive_cultures = positive_cultures.drop("MUCOR SP.", axis=0)

In [68]:
positive_cultures = positive_cultures.drop("MYCOPLASMA HOMINIS", axis=0)

In [69]:
positive_cultures = positive_cultures.drop("ORGANISM", axis=0)

In [70]:
positive_cultures = positive_cultures.drop("PARAINFLUENZA VIRUS TYPE 1", axis=0)

In [71]:
positive_cultures = positive_cultures.drop("POSITIVE FOR CYTOMEGALOVIRUS", axis=0)

In [72]:
positive_cultures = positive_cultures.drop("RHINOVIRUS", axis=0)

In [73]:
positive_cultures = positive_cultures.drop("VARICELLA-ZOSTER VIRUS", axis=0)

In [74]:
positive_cultures = positive_cultures.drop("VIRUS", axis=0)

In [75]:
positive_cultures =positive_cultures.reset_index()

In [76]:
positive_cultures

,org_name,spec_type,subject_id,hadm_id,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
0,YEAST,SPUTUM,3,145834,NaN,NaN,76.52,M,Unspecified septicemia,70062.0,1.0,80075.0,NaN
1,YEAST,URINE,3,145834,NaN,NaN,76.52,M,Unspecified septicemia,70079.0,1.0,80075.0,NaN
2,YEAST,URINE,3,145834,NaN,NaN,76.52,M,Unspecified septicemia,70079.0,1.0,80075.0,NaN
3,GRAM NEGATIVE ROD(S),SWAB,3,145834,NaN,NaN,76.52,M,Unspecified septicemia,70070.0,1.0,80058.0,NaN
4,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",SWAB,3,145834,NaN,NaN,76.52,M,Unspecified septicemia,70070.0,1.0,80155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262676,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,MRSA SCREEN,99991,151118,NaN,NaN,47.72,M,Severe sepsis,70091.0,1.0,80293.0,NaN
262677,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,MRSA SCREEN,99991,151118,NaN,NaN,47.72,M,Severe sepsis,70091.0,1.0,80293.0,NaN
262678,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,MRSA SCREEN,99991,151118,NaN,NaN,47.72,M,Unspecified septicemia,70091.0,1.0,80293.0,NaN
262679,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,MRSA SCREEN,99991,151118,NaN,NaN,47.72,M,Unspecified septicemia,70091.0,1.0,80293.0,NaN


# SAVING NEW DATAFRAMA 
## POSITIVE CULTURES ADJUSTED
### Drop diagnoses; Rename spec_type

In [77]:
positive_cultures_ajusted = positive_cultures.to_csv('positive_cultures_ajustes.csv', index=False)

In [78]:
positive_cultures_ajusted = positive_cultures

In [79]:
type(positive_cultures_ajusted )

pandas.core.frame.DataFrame

## CREATING NEW DATAFRAME (BLOOD CULTURES)

### BLOOD CULTURES POSITIVES

In [83]:
blood_cultures_positives = positive_cultures_ajusted [positive_cultures_ajusted.spec_type == 'BLOOD CULTURE']

In [84]:
blood_cultures_positives

,org_name,spec_type,subject_id,hadm_id,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
8,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,LINEZOLID,S,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90031.0
9,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,AMPICILLIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90004.0
10,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,NaN,NaN,87.82,M,Other specified septicemias,70012.0,1.0,80168.0,NaN
11,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,LEVOFLOXACIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90025.0
12,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,VANCOMYCIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262561,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,PENICILLIN G,R,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90003.0
262562,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,VANCOMYCIN,R,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90015.0
262563,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,LINEZOLID,S,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90031.0
262564,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,DAPTOMYCIN,S,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90030.0


In [85]:
blood_cultures_positives.to_csv('blood_cultures_positives.csv', index=False)

# EXPLORATORY ANALYSIS

# POSITIVE CULTURES 
## df positive_cultures_ajusted

In [2]:
# positive_cultures_ajusted = pd.read_csv('positive_cultures_ajustes.csv')

In [86]:
# How many patients (80,1%)
positive_cultures_ajusted['subject_id'].nunique()

4442

In [78]:
# How many admissions
positive_cultures_ajusted['hadm_id'].nunique()

5044

In [79]:
# How many microorganisms
positive_cultures_ajusted['org_name'].nunique()

226

In [80]:
# How many specimen types
positive_cultures_ajusted['spec_type'].nunique()

29

In [81]:
# How many tests (cultures)
positive_cultures_ajusted['org_itemid'].nunique()

207

# BLOOD CULTURES
## df blood culture positive

In [3]:
# blood_cultures_positives = pd.read_csv('blood_cultures_positives.csv')

In [91]:
blood_cultures_positives

,org_name,spec_type,subject_id,hadm_id,ab_name,interpretation,age,gender,diagnoses,spec_itemid,isolate_num,org_itemid,ab_itemid
8,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,LINEZOLID,S,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90031.0
9,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,AMPICILLIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90004.0
10,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,NaN,NaN,87.82,M,Other specified septicemias,70012.0,1.0,80168.0,NaN
11,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,LEVOFLOXACIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90025.0
12,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,21,111970,VANCOMYCIN,R,87.82,M,Other specified septicemias,70012.0,2.0,80168.0,90015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262561,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,PENICILLIN G,R,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90003.0
262562,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,VANCOMYCIN,R,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90015.0
262563,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,LINEZOLID,S,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90031.0
262564,ENTEROCOCCUS FAECIUM,BLOOD CULTURE,99836,101018,DAPTOMYCIN,S,52.10,F,Streptococcal septicemia,70012.0,1.0,80168.0,90030.0


In [93]:
# 2477 (27,75%) patientis with positive blood culture
blood_cultures_positives['hadm_id'].nunique()

2477